In [ ]:
import os
import requests
import urllib
import numpy as np
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import asyncio
from data_quality_utils import Crawler
from sentence_transformers import SentenceTransformer, util
from typing import List, Optional, Union
from statistics import mode
from crawl4ai import AsyncWebCrawler, CrawlerRunConfig, CacheMode
from crawl4ai.deep_crawling import BFSDeepCrawlStrategy, BestFirstCrawlingStrategy
from crawl4ai.content_scraping_strategy import LXMLWebScrapingStrategy
from crawl4ai.deep_crawling.scorers import KeywordRelevanceScorer
from crawl4ai.deep_crawling.filters import FilterChain, URLPatternFilter, ContentRelevanceFilter, SEOFilter, ContentTypeFilter
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
from crawl4ai.content_filter_strategy import BM25ContentFilter
from sentence_transformers import SentenceTransformer, util
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# embedding model
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# suppress warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# get data from datasette
datasette_base_url = "https://datasette.planning.data.gov.uk/digital-land.csv"

query = """
select * 
from source as s
left join organisation as o
on s.organisation=o.organisation 
where s.collection = "conservation-area"
"""
encoded_query = urllib.parse.urlencode({"sql": query})

r = requests.get(f"{datasette_base_url}?{encoded_query}", auth=('user', 'pass'))

filename = "datasette_data.csv"
with open(filename, "wb") as f_out:
    f_out.write(r.content)

In [ ]:
# group by organisation as we're looking for one page per council
data = (
    pl.read_csv(filename)
    .group_by("name")
    .agg(pl.col("website").first(), pl.col("documentation_url"))
)
data = data.with_columns(pl.col("website").str.strip_chars_end("/"))
data

In [ ]:
def split_text(text, chunk_size, chunk_overlap, separators, keep_separator):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,  
        separators=separators,
        keep_separator=keep_separator
    )

    chunks = text_splitter.split_text(text)
    return chunks

In [ ]:
def chunk_processing(crawl_data, chunk_size, chunk_overlap, separators, keep_separator):
    chunking_data = list()
    
    for url, markdown in crawl_data:
        chunk_texts = split_text(markdown, chunk_size, chunk_overlap, separators, keep_separator)
        for chunk_num, chunk in enumerate(chunk_texts):
            chunk_id = f"{url}-{chunk_num}" # To have unique id for data.
            chunking_data.append((chunk_id, url, chunk_num, chunk))
    
    chunked_df = pl.DataFrame(chunking_data, 
                              schema=["id", "url", "chunk_num", "text"], orient="row")
    
    embeddings = embedding_model.encode(chunked_df['text'].to_list(), convert_to_numpy=True)
    chunked_df = chunked_df.with_columns(pl.Series(name="embedding", values=embeddings))
    
    return chunked_df

In [ ]:
def basic_processing(crawl_data):
    basic_data = list()
    
    for url, markdown in crawl_data: 
        basic_data.append((f"{url}-0", url, 0, markdown))
    
    basic_df = pl.DataFrame(basic_data, 
                            schema=["id", "url", "chunk_num", "text"], orient="row")
    
    embeddings = embedding_model.encode(basic_df['text'].to_list(), convert_to_numpy=True)
    basic_df = basic_df.with_columns(pl.Series(name="embedding", values=embeddings))

    return basic_df

In [ ]:
def get_similar_urls(
        crawl_data, 
        prompt, 
        num_results, 
        chunking,
        chunk_size, 
        chunk_overlap, 
        separators, 
        keep_separator,
        ):
    """
    Embeds crawled webpage data, computes similarity to a given prompt, and returns the top N most similar pages.

    Parameters:
    - crawl_data (list of tuples): A list of tuples containing (url, markdown) for each crawled page.
    - prompt (str): The text prompt to compare against the crawled page embeddings.
    - num_results (int, optional): The number of top similar pages to return. If None, returns all pages.
    - chunking (bool, optional): Whether we want to embed the full webpage or chunk.

    Returns:
    - polars.DataFrame: A DataFrame containing:
        - "url": The webpage URL.
        - "markdown": The extracted markdown content.
        - "embedding": The computed embedding for the content.
        - "similarity": The cosine similarity score with the prompt.
    """
    res = []
    
    if chunking:
        crawl_df = chunk_processing(
            crawl_data = crawl_data,
            chunk_size = chunk_size, 
            chunk_overlap = chunk_overlap, 
            separators = separators, 
            keep_separator = keep_separator
        )
    elif not chunking:
        crawl_df = basic_processing(crawl_data)
    
    embeddings = np.stack(crawl_df["embedding"].to_list())
    
    prompt_embedding = np.array(embedding_model.encode(prompt, convert_to_numpy=True), dtype='float64')
    
    # get similarity scores
    sim=util.cos_sim(
        prompt_embedding.astype(np.float32), 
        embeddings.astype(np.float32)
    )
    # get indices of top n most similar urls
    if not num_results:
        num_results = len(crawl_df)
    indices = np.argsort(sim).numpy().flatten()[:-num_results-1:-1]
    sorted_df = (
        crawl_df[indices]
        .with_columns(similarity=np.sort(sim).flatten()[:-num_results-1:-1])
    )
    return sorted_df

In [ ]:
def pretty_print_results(sorted_df, num_printing_results):
    # print top n urls with similarity scores
    print("\nTop Similar Pages:\n" + "="*40)
    for i in range(min(num_printing_results, len(sorted_df))):
        url = sorted_df.get_column("url")[i]
        score = sorted_df.get_column("similarity")[i]
        print(f"{i+1}. {url.ljust(60)} | Similarity: {score:.4f}")

In [ ]:
def load_csv_data(file_path):
  pl.Config.set_float_precision(10)
  # To give some more readiability, regardless of what is shown the numbers ARE the same!
  df = pl.read_csv(file_path)
  cleaned_df = df.with_columns(
      pl.col("embedding")
        .str.strip_chars("[]")
        .str.replace_all("\n", '')
        .str.strip_chars(" ")
        .str.split(' ')
        .list.eval(pl.element().filter(pl.element() != ""))
        .list.eval(pl.element().cast(pl.Float32, strict=False))
  )
  return cleaned_df

In [ ]:
async def process_council(
    council_names,
    max_depth=6,
    keyword_scorer=None,
    filters=None,
    prompt="A page about conservation areas.",
    cache_enabled=False,
    num_results=None,
    num_printing_results=10,
    save_dfs = False,
    load_dfs = False,
    data_dir = '',
    chunking = False,
    chunk_size = 300, 
    chunk_overlap = 50, 
    separators = ["##", "\n\n", ". ", " ", ""], 
    keep_separator = False,
):
    crawler = Crawler(
            max_depth=max_depth,
            keyword_scorer=keyword_scorer,
            filters=filters,
            cache_enabled=cache_enabled,
        )
    
    crawled_dfs = dict()

    for council_name in council_names:
        council_data = data.filter(pl.col("name").str.contains(council_name))
        short_council_name = council_data.get_column("website")[0].split('.')[1]
        full_name = council_data.get_column("name")[0]
        homepage = council_data.get_column("website")[0]
        prompt = prompt.format((full_name).replace('\n', ''))
        save_path = f"{data_dir}{short_council_name}.csv"

        print("="*40 + f"\nProcessing {full_name}...\n")
        
        if os.path.isdir(data_dir) and load_dfs:
            if f"{short_council_name}.csv" in os.listdir(data_dir): 
                sorted_df = load_csv_data(save_path)
                pretty_print_results(sorted_df, num_printing_results)
                crawled_dfs[homepage] = sorted_df
                continue
            
        # crawl url
        crawl_data = await crawler.deep_crawl(homepage)
        
        # get markdown embeddings
        sorted_df = get_similar_urls(
            crawl_data = crawl_data, 
            prompt = prompt,
            num_results=num_results, 
            chunking=chunking,
            chunk_size = chunk_size, 
            chunk_overlap = chunk_overlap, 
            separators = separators, 
            keep_separator = keep_separator
        )
        
        sorted_df = sorted_df.with_columns(pl.col("url").str.strip_chars_end("/"))
        
        pretty_print_results(sorted_df, num_printing_results)

        if save_dfs:
            save_df = sorted_df.to_pandas()
            save_df.to_csv(save_path)

        crawled_dfs[homepage] = sorted_df
    
    return crawled_dfs

## Our approach

Our approach involves 2 main steps: a web crawler and an embedding similarity search. Below is a description of these steps.

### Web crawler

The web crawler takes a homepage URL of an organisation (council website) and crawls it to look for pages talking about conservation areas.

The crawler will look for links on a single page, put them in a queue and then iteratively check them until it finds what it was looking for or it reaches a stopping criterion, such as maximum depth (how many clicks away from home page). 

In order to save time, we can define some scorers or filters which tell the crawler which pages to prioritise or ignore. In this case, some common patterns of what a user needs to click to get to the page of interest are _"planning"_, _"building"_, _"heritage"_ or _"conservation"_.

The crawler uses a *"best first strategy"*, which utilises the scorers or filters to visit most relevant sites first, rather than a depth-first or breath-first search.

The crawler extracts the HTML from the pages and turns them into markdown. This is because it's more readable and easier to work with in the next steps. The crawler returns a list of pairs of (_url_, _markdown_).

### Embedding search

To be filled

### The next few cells show how to use the tools to find conservation area pages.

You can define your own parameters, such as maximum depth, how many results you want to see and any scorers or filters. Below is a template showing how to defin each scorer/filter type correctly - all you need to do is change the keywords or patterns.

You can also define a prompt - this is what will be used to get embeddings scores for a webpage. The more similar the prompt is to what a conservation area page usually looks like, the more accurate the results.

Lastly, you can await the `process_council` function, which will run the functionality described above and print the results. You can use it for one council only or for a list of councils.

#### Template for how to define filters or scorers
##### Pick the types you need and adjust their parameter (keywords, threshold, ...) and pass them to the crawler function.
    keyword_scorer = {
        "keywords": ["conservation", "conservation area", "planning", "building", "urban", "heritage", "resident"],
        "weight": 0.8,
    }
        
    filters=[
        {"type": "SEOFilter", "threshold": 0.6, "keywords": ["conservation", "area", "planning", "heritage", "resident"]},
        {"type": "ContentRelevanceFilter", "query": "conservation area or planning data", "threshold": 0.2},
        {"type": "ContentTypeFilter", "allowed_types": ["text/html"]},
        {"type": "URLPatternFilter", "patterns": ["*conservation*", "*planning*", "*building*"]},
    ]

### Gedling

In [ ]:
max_depth = 6
num_results = 100
num_printing_results = 10
filters=[
    {"type": "ContentTypeFilter", "allowed_types": ["text/html"]},
    {"type": "URLPatternFilter", "patterns": ["*conservation*", "*planning*", "*building*"]},
]

# please write the prompt such that there is a curly bracket where the council
# name will be inserted
prompt = """
The text discusses conservation areas from the {} and includes data on 
planning data, areas, interactive maps, appraisals, notices, boundaries, 
links and similar.
"""

gedling_dict = await process_council(
    council_names=["Gedling"], 
    max_depth=max_depth, 
    filters=filters, 
    prompt=prompt, 
    num_results=num_results, 
    num_printing_results=num_printing_results
)

### South Gloucestershire

In [ ]:
max_depth = 6
num_results = 100
num_printing_results = 10
filters=[
    {"type": "ContentTypeFilter", "allowed_types": ["text/html"]},
    {"type": "URLPatternFilter", "patterns": ["*conservation*", "*planning*", "*building*"]},
]

# please write the prompt such that there is a curly bracket where the council
# name will be inserted
prompt = """
The text discusses conservation areas from the {} and includes data on 
planning data, areas, interactive maps, appraisals, notices, boundaries, 
links and similar.
"""

sglos_dict = await process_council(
    council_names=["South Gloucestershire"], 
    max_depth=max_depth, 
    filters=filters, 
    prompt=prompt, 
    num_results=num_results
    num_printing_results=num_printing_results
)

### Bournemouth, Christchurch and Poole

In [ ]:
max_depth = 6
num_results = 100
num_printing_results = 10
filters=[
    {"type": "ContentTypeFilter", "allowed_types": ["text/html"]},
    {"type": "URLPatternFilter", "patterns": ["*conservation*", "*planning*", "*building*"]},
]

# please write the prompt such that there is a curly bracket where the council
# name will be inserted
prompt = """
The text discusses conservation areas from the {} and includes data on 
planning data, areas, interactive maps, appraisals, notices, boundaries, 
links and similar.
"""

bmccpl_dict = await process_council(
    council_names=["Bournemouth, Christchurch and Poole"], 
    max_depth=max_depth, 
    filters=filters, 
    prompt=prompt, 
    num_results=num_results, 
    num_printing_results=num_printing_results
)

### Warrington

In [ ]:
max_depth = 6
num_results = 100
num_printing_results = 10
filters=[
    {"type": "ContentTypeFilter", "allowed_types": ["text/html"]},
    {"type": "URLPatternFilter", "patterns": ["*conservation*", "*planning*", "*building*"]},
]

# please write the prompt such that there is a curly bracket where the council
# name will be inserted
prompt = """
The text discusses conservation areas from the {} and includes data on 
planning data, areas, interactive maps, appraisals, notices, boundaries, 
links and similar.
"""

warr_dict = await process_council(
    council_names=["Warrington"], 
    max_depth=max_depth, 
    filters=filters, 
    prompt=prompt, 
    num_results=num_results, 
    num_printing_results=num_printing_results
)

### Stoke on Trent

In [ ]:
max_depth = 6
num_results = 100
num_printing_results = 10
filters=[
    {"type": "ContentTypeFilter", "allowed_types": ["text/html"]},
    {"type": "URLPatternFilter", "patterns": ["*conservation*", "*planning*", "*building*"]},
]

# please write the prompt such that there is a curly bracket where the council
# name will be inserted
prompt = """
The text discusses conservation areas from the {} and includes data on 
planning data, areas, interactive maps, appraisals, notices, boundaries, 
links and similar.
"""

stoke_dict = await process_council(
    council_names=["Stoke"], 
    max_depth=max_depth, 
    filters=filters, 
    prompt=prompt, 
    num_results=num_results,
    num_printing_results=num_printing_results
)

### Redbridge

In [ ]:
max_depth = 6
num_results = 100
num_printing_results = 10
filters=[
    {"type": "ContentTypeFilter", "allowed_types": ["text/html"]},
    {"type": "URLPatternFilter", "patterns": ["*conservation*", "*planning*", "*building*"]},
]

# please write the prompt such that there is a curly bracket where the council
# name will be inserted
prompt = """
The text discusses conservation areas from the {} and includes data on 
planning data, areas, interactive maps, appraisals, notices, boundaries, 
links and similar.
"""

redbridge_dict = await process_council(
    council_names=["Redbridge"], 
    max_depth=max_depth, 
    filters=filters, 
    prompt=prompt, 
    num_results=num_results, 
    num_printing_results=num_printing_results
)

### York

In [ ]:
max_depth = 6
num_results = 100
num_printing_results = 10
filters=[
    {"type": "ContentTypeFilter", "allowed_types": ["text/html"]},
    {"type": "URLPatternFilter", "patterns": [r"*[Cc]onservation*", r"*[Pp]lanning*", r"*[Bb]uilding*"]},
]

# please write the prompt such that there is a curly bracket where the council
# name will be inserted
prompt = """
The text discusses conservation areas from the {} and includes data on 
planning data, areas, interactive maps, appraisals, notices, boundaries, 
links and similar.
"""

york_dict = await process_council(
    council_names=["York"], 
    max_depth=max_depth, 
    filters=filters, 
    prompt=prompt, 
    num_results=num_results, 
    num_printing_results=num_printing_results
)

### Malvern Hills

In [ ]:
max_depth = 6
num_results = 100
num_printing_results = 10
filters=[
    {"type": "ContentTypeFilter", "allowed_types": ["text/html"]},
    {"type": "URLPatternFilter", "patterns": [r"*[Cc]onservation*", r"*[Pp]lanning*", r"*[Bb]uilding*"]},
]

# please write the prompt such that there is a curly bracket where the council
# name will be inserted
prompt = """
The text discusses conservation areas from the {} and includes data on 
planning data, areas, interactive maps, appraisals, notices, boundaries, 
links and similar.
"""

malvern_dict = await process_council(
    council_names=["Malvern Hills"], 
    max_depth=max_depth, 
    filters=filters, 
    prompt=prompt, 
    num_results=num_results, 
    num_printing_results=num_printing_results
)

## Multiple councils

You can define any list of councils and their processing will be executed sequentially

In [ ]:
np.random.seed(4321)
num_examples = 10
example_idx = np.random.randint(0, len(data), num_examples)
examples = data[example_idx]
examples

In [ ]:
max_depth = 6
num_results = 100
num_printing_results = 10
filters=[
    {"type": "ContentTypeFilter", "allowed_types": ["text/html"]},
    {"type": "URLPatternFilter", "patterns": ["*conservation*", "*planning*", "*building*"]},
]

# please write the prompt such that there is a curly bracket where the council
# name will be inserted
prompt = """
The text discusses conservation areas from the {} and includes data on 
planning data, areas, interactive maps, appraisals, notices, boundaries, 
links and similar.
"""

multi_dict = await process_council(
    council_names=examples.get_column("name"), 
    max_depth=max_depth, 
    filters=filters, 
    prompt=prompt, 
    num_results=num_results,
    num_printing_results=num_printing_results
)

## Evaluating Results with Chunking

For this section we manually labelled 25 councils with their true page. We perform tests and ranking here to assess the performance of our model. First we import a manually defined list of true documentation urls as test_df, and filter our main dataset for that. We also strip the final slash and will clean urls in other ways throughout.

In [ ]:
test_df = pl.read_csv("data/page_ranking_truth_df.csv")
test_df = test_df.with_columns(pl.col("correct_documentation_url").str.strip_chars_end("/"))
test_website_list = test_df["website"].to_list()
data_filtered = data.filter(pl.col("website").is_in(test_website_list))
combined_df = data_filtered.join(test_df, on='website')

The next cell parses all councils in our test set. This takes some time to run, but there is functionality for saving and loading data.

In [ ]:
max_depth = 6
num_results = 100
num_printing_results = 10
filters=[
    {"type": "ContentTypeFilter", "allowed_types": ["text/html"]},
    {"type": "URLPatternFilter", "patterns": [r"*[Cc]onservation*", r"*[Pp]lanning*", r"*[Bb]uilding*"]},
]

# please write the prompt such that there is a curly bracket where the council
# name will be inserted
prompt = """
Conservation Areas in {}

We are committed to preserving the historic and architectural character of our borough
through designated conservation areas. These areas protect buildings, streets, and 
landscapes of special significance, ensuring that any changes respect their unique heritage. 
If you live or own property in a conservation area, additional planning controls may 
apply to alterations, demolitions, and new developments. Our aim is to balance modern 
needs with the protection of our historic environment. For more information on conservation 
area guidelines, planning applications, and how you can contribute to local heritage preservation, 
please visit our planning and conservation pages. You will find maps, appraisal documents and the
list of council conservation areas. 
"""
# Fair warning, this will take multiple hours to run in its entirety. 

test_output_dict = await process_council(
    council_names=combined_df.get_column("name"), # Just the first 5 takes 20 mins.
    max_depth=max_depth, 
    filters=filters,        
    prompt=prompt, 
    num_results=num_results,
    num_printing_results=num_printing_results,
    # save_dfs = True, # Uncomment if you want to save your parsed table. This will overwrite anything in the data_dir, so do change if needed.
    # load_dfs = True, # Uncomment if you want to quickly load previous parses from data_dir. This will lead to no scraping being done.
    # data_dir = 'data/chunking_csvs/', # Uncomment to specify save/load location.
    chunking = True, # Set to False if you would like the entire webpage embedded, not chunks.
    chunk_size=500,
    chunk_overlap=75
)

In [ ]:
def url_cleaning(df, col):
    df = df.with_columns(pl.col(col).str.replace(
        "http://", "https://", literal=True))
    df = df.with_columns(pl.col(col).str.replace(
        "www.", "", literal=True))
    df = df.with_columns(pl.col(col).str.strip_chars_end("/"))
    return df

In [ ]:
def remove_duplicates(df):
    df = url_cleaning(df, 'url')
    df = url_cleaning(df, 'id')

    df = df.sort("similarity", descending=True)
    # We only want the best chunk (if chunking) from each webpage.
    df = df.group_by("url", maintain_order=True).first()
    df = df.sort(by = 'similarity', descending=True)

    return df

In [ ]:
def get_result_rank(correct_webpage, df):
    # Here we get an ordered list of the similarity scores, along with the best one.
    # Note, multiple rows may have the same similarity.
    similarity_scores = list(set(df['similarity']))
    sorted_similarity_scores = sorted(similarity_scores, reverse=True)
    result_rows = df.filter(pl.col('url') == correct_webpage)
    best_similarity_score = result_rows['similarity'][0]

    # Now we acquire the rank of the embedding score achieved by our pipeline.
    rank = sorted_similarity_scores.index(best_similarity_score) + 1
    print(f'rank is {rank} with similarity {best_similarity_score}')
    return rank

In [ ]:
def test_ranking_processing(test_df: pl.dataframe.frame.DataFrame, test_output_dict: dict, outlier_threshold: int = 1000):
    """Processing to analyse the ranking of our pipeline's embedding guesses. 

    :param test_df: polars.DataFrame with columns:
        - "website": The base website of the council
        - "correct_documentation_url": The manually judged correct webpage. This should have the largest embedding similarity.
    :param test_output_dict: dictionary with outputs of webscraping and embedding. Keys are base websites and values are dataframes.
    :param outlier_threshold: Threshold for ranks beyond which results are assumed false/failure. If above, not saved in ranking_dict.
    :return: 
        ranking_dict: Dictionary with ranks for each website. Base websites are keys, ranks are values
        num_unclassified: Quantity of failed experiments, where no website match could be found.
        num_outliers: Quantity of experiments leading to outliers.
    """
    
    ranking_dict = dict()
    filtered_output_dict = {url: remove_duplicates(df)
                        for url, df in test_output_dict.items()}
    
    num_unclassified = 0
    num_outliers = 0

    for base_website, similarity_df in filtered_output_dict.items():
        print(f'processing {base_website}')
        try:
            correct_webpage = test_df.filter(pl.col('website') == base_website)['correct_documentation_url'][0]
            rank = get_result_rank(correct_webpage, similarity_df)
            if rank >= outlier_threshold:
                print(f"Outlier for this url - rank of {rank} - leaving out of plot")
                num_outliers += 1
                continue
            
            ranking_dict[base_website] = rank
        
        except IndexError as e:
            print(f"No match for correct website via urls - error message '{e}'")
            num_unclassified += 1 
                
    return ranking_dict, num_unclassified, num_outliers

In [ ]:
test_df = url_cleaning(test_df, 'correct_documentation_url')
ranking_dict, num_unclassified, num_outliers = test_ranking_processing(test_df, 
                                                                       test_output_dict, 
                                                                       outlier_threshold=30)

In [ ]:
ranks = list(ranking_dict.values())
height = ranks.count(mode(ranks))

ax = sns.histplot(x=ranks, binwidth=1, discrete=True, color='#00625E')
ax.set_title(f'Frequency of Ranks - {num_unclassified} unclassified, {num_outliers} outlier/s')
ax.set_xlabel('Rank')
ax.set_ylabel('Quantity')
ax.set_xticks(np.arange(1, max(ranks)+1))
ax.set_yticks(np.arange(0, height+1, 2))
ax.set_xticklabels([str(num) for num in np.arange(1, max(ranks)+1)])
ax.set_yticklabels([str(num) for num in np.arange(0, height+1, 2)])

plt.tight_layout()
plt.show()

In [ ]:
def mean_reciprocal_rank(ranks):
    size = len(ranks)
    return sum([1/rank for rank in ranks])/size

In [ ]:
mrr = mean_reciprocal_rank(ranks)
print(f"Mean Reciprocal Rank of - {mrr}")

### Case Studies

Let's look a little further at the ones that did not work out, starting with Camden. We see that the correct page is not scraped

In [ ]:
filtered_output_dict = {url: remove_duplicates(df)
                        for url, df in test_output_dict.items()}

correct_page = test_df.filter(
    pl.col('website') == 'https://www.camden.gov.uk'
)['correct_documentation_url'][0]

print(filtered_output_dict[
    'https://www.camden.gov.uk'
].filter(pl.col("url") == correct_page)['text'])

In [ ]:
correct_page

The page above (that is, one click away from the ideal one) is parsed, but the crawler struggles with security on the site.

In [ ]:
page_above = 'https://camden.gov.uk/conservation-areas'
print(filtered_output_dict[
    'https://www.camden.gov.uk'
].filter(pl.col("url") == page_above)['text'][0])

Below is the code from the highest similarity chunk, taken from one of the boxes on https://camden.gov.uk/planning-building-development. 

In [ ]:
print(filtered_output_dict[
    'https://www.camden.gov.uk'
]['text'][0])

For Buckinghamshire, we see a few relevant links, the top 4 listed below. We assume https://buckinghamshire.gov.uk/planning-and-building-control/heritage/conservation-areas-in-buckinghamshire is correct. 

In [ ]:
for r in filtered_output_dict['https://www.buckinghamshire.gov.uk']['url'][0,4,7,13]:
    print(r)

For West Berkshire, the top three are very close in terms of similarity rating. We assume https://westberks.gov.uk/conservationareas is the correct one, with the other two having highly relevant snippets of information on the page. 

In [ ]:
for r in filtered_output_dict['https://www.westberks.gov.uk']['url', 'similarity'][:3].iter_rows():
    print(r)

For South Staffordshire, the true url (https://www.sstaffs.gov.uk/planning/conservation-and-heritage/south-staffordshires-conservation-areas) is very light on the detail usually present. The closest match (https://www.sstaffs.gov.uk/planning/conservation-and-heritage/conservation-areas) is one click away.

In [ ]:
for r in test_output_dict['https://www.sstaffs.gov.uk']['url'][0,7,11,30]:
    print(r)

For Rochdale, the answer was not in the usual first 100 chunks. As such, here we run a full test keeping every single chunk.

In [ ]:
max_depth = 6
num_printing_results = 10
filters=[
    {"type": "ContentTypeFilter", "allowed_types": ["text/html"]},
    {"type": "URLPatternFilter", "patterns": [r"*[Cc]onservation*", r"*[Pp]lanning*", r"*[Bb]uilding*"]},
]

# please write the prompt such that there is a curly bracket where the council
# name will be inserted
prompt = """
Conservation Areas in {}

We are committed to preserving the historic and architectural character of our borough
through designated conservation areas. These areas protect buildings, streets, and 
landscapes of special significance, ensuring that any changes respect their unique heritage. 
If you live or own property in a conservation area, additional planning controls may 
apply to alterations, demolitions, and new developments. Our aim is to balance modern 
needs with the protection of our historic environment. For more information on conservation 
area guidelines, planning applications, and how you can contribute to local heritage preservation, 
please visit our planning and conservation pages. You will find maps, appraisal documents and the
list of council conservation areas. 
"""

rochdale_dict = await process_council(
    council_names=['Rochdale'],
    max_depth=max_depth, 
    filters=filters,        
    prompt=prompt, 
    num_printing_results=num_printing_results,
    chunking = True,
    chunk_size=500,
    chunk_overlap=75
)

With this we see that the webpage was crawled, but something went wrong, presumably due to too much scraping being detected.

In [ ]:
filtered_rochdale_dict = {url: remove_duplicates(df)
                        for url, df in rochdale_dict.items()}

correct_page = test_df.filter(
    pl.col('website') == 'https://www.rochdale.gov.uk'
)['correct_documentation_url'][0]

filtered_rochdale_dict[
    'https://www.rochdale.gov.uk'
].filter(pl.col('url') == correct_page)['text'][0]

For Dover, the web scraper simply does not parse the website properly. It only crawls over two distinct pages, the home page and the planning page.

In [ ]:
filtered_output_dict['https://www.dover.gov.uk']['url'].to_list()

This may be down to the .aspx extension used for the website, so the web crawler may not be appropriate in this case.